In [21]:
# https://cnpnote.tistory.com/entry/PYTHON-%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%97%90%EC%84%9C-selenium-webdriver%EB%A5%BC-%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC-%EC%9B%B9-%ED%8E%98%EC%9D%B4%EC%A7%80%EB%A5%BC-%EC%8A%A4%ED%81%AC%EB%A1%A4%ED%95%98%EB%8A%94-%EB%B0%A9%EB%B2%95%EC%9D%80-%EB%AC%B4%EC%97%87%EC%9E%85%EB%8B%88%EA%B9%8C
# https://code-ing.tistory.com/6
# https://cnpnote.tistory.com/entry/PYTHON-파이썬에서-selenium-webdriver를-사용하여-웹-페이지를-스크롤하는-방법은-무엇입니까

# 라이브러리 임포트

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import time

import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 태그명별 포스트 URL 크롤링

In [17]:
# Chrome Driver
driver = webdriver.Chrome("C:/shkim/util/chromedriver/chromedriver.exe")

from urllib import parse
TAG_NAME = "ㄱㅂㅇ"
url_tmp = "www.instagram.com/explore/tags/" + TAG_NAME
url = "https://" + parse.quote(url_tmp) # url주소에 포함된 한글 처리

# 인스타그램 태그 사이트 접속
driver.get(url)

time.sleep(2)


SCROLL_PAUSE_TIME = 1.6

urlList = [] 

last_height = ""
new_height = ""

while True :
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    
    #body = driver.find_element_by_tag_name("body")
    #body.send_keys(Keys.PAGE_DOWN)
    
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME) # 인터넷 속도도 중요하다..
    
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    print('last height : ', last_height, ", new height : ", new_height)
    
    if new_height != last_height :
        
        # 개별 이미지 div 리스트
        results = driver.find_elements_by_css_selector('div.v1Nh3')
        
        for i in range(len(results)) :
            # 개별 이미지의 링크 획득
            link = results[i].find_element_by_tag_name('a')
            href = link.get_attribute('href')
            urlList.append(href)        
        
        last_height = new_height
        continue
    else :
        
        # 개별 이미지 div 리스트
        results = driver.find_elements_by_css_selector('div.v1Nh3')
        
        for i in range(len(results)) :
            # 개별 이미지의 링크 획득
            link = results[i].find_element_by_tag_name('a')
            href = link.get_attribute('href')
            urlList.append(href)
        
        break

last height :  4166 , new height :  5450
last height :  5450 , new height :  6734
last height :  6734 , new height :  8018
last height :  8018 , new height :  9302
last height :  9302 , new height :  10586
last height :  10586 , new height :  11870
last height :  11870 , new height :  13154
last height :  13154 , new height :  14438
last height :  14438 , new height :  15722
last height :  15722 , new height :  16685
last height :  16685 , new height :  18290
last height :  18290 , new height :  19253
last height :  19253 , new height :  20858
last height :  20858 , new height :  21821
last height :  21821 , new height :  23426
last height :  23426 , new height :  24068
last height :  24068 , new height :  25352
last height :  25352 , new height :  25264
last height :  25264 , new height :  25264


In [18]:
# 중복 제거
print("Before deduplication : ", len(urlList))
urlList = list(set(urlList))
print("After deduplication : ", len(urlList))

Before deduplication :  933
After deduplication :  231


# 각 포스트별 내용 가져오기

In [40]:
driver = webdriver.PhantomJS("C:/shkim/util/phantomjs-2.1.1-windows/bin/phantomjs.exe")

post_dateList = [] # 일자
post_tpList = [] # 포스트 유형 (image, video)
like_cntList = [] # 좋아요 갯수
view_cntList = [] # 비디오 조회수
cmt_cntList = [] # 댓글 갯수
contentList = [] # 글 내용
hashtagsList = [] # 해시태그 (","으로 연결)

for i in range(len(urlList)) :
    
    print((i+1), " : ", urlList[i], " start...")
    
    driver.get(urlList[i])
    
    driver.implicitly_wait(2)
    
    try :
        # 포스팅 날짜(post_date)
        post_dateArea1 = driver.find_element_by_css_selector('time._1o9PC')
        post_dateArea2 = post_dateArea1.get_attribute('datetime')
        post_date = post_dateArea2[:10].replace("-","")
        post_dateList.append(post_date)

        ##########################################################################
        
        post_tp = ""

        likeCheck = driver.find_elements_by_css_selector('div.Nm9Fw') # 좋아요 여부, 0이면 좋아요 개수가 0이거나 비디오
        imgYn = driver.find_elements_by_css_selector('div._9AhH0') # likeCheck가 0 인데 imgYn이 0이 아니면 좋아요 갯수가 0개이다.
        viewCheck = driver.find_elements_by_css_selector('span.vcOH2') # 조회 여부, 0이면 조회수가 0이거나 이미지
        vdoYn = driver.find_elements_by_css_selector('a.QvAa1') # viewCheck가 0인데 vdoYn이 0이 아니면 조회수가 0회이다.

        if (len(likeCheck) == 0) & (len(viewCheck) == 0) : # 좋아요, 조회수가 없는 경우
            if len(imgYn) != 0 : # 사진의 좋아요가 0
                post_tp = "image"
                like_cntList.append("0")
                view_cntList.append("0")

            elif len(vdoYn) != 0 : # 비디오의 조회수가 0
                post_tp = "video"
                like_cntList.append("0")
                view_cntList.append("0")

        elif len(likeCheck) != 0 : # 좋아요 있음
            # 좋아요 갯수 (like_cnt)
            post_tp = "image"
            like_cntArea1 = driver.find_element_by_css_selector('div.Nm9Fw')
            like_cntArea2 = like_cntArea1.find_element_by_tag_name('span')
            like_cnt = like_cntArea2.text
            like_cntList.append(like_cnt)
            view_cntList.append("0")   

        elif len(viewCheck) != 0 : # 조회수 있음
            # 조회수 (view_cnt)
            post_tp = "video"
            view_cntArea1 = driver.find_element_by_css_selector("span.vcOH2")
            view_cntArea2 = view_cntArea1.find_element_by_tag_name('span')
            view_cnt = view_cntArea2.text
            like_cntList.append("0")
            view_cntList.append(view_cnt)         


        # 포스트 유형 (post_tp)        
        post_tpList.append(post_tp)
            
        ##########################################################################
        
        # 댓글 갯수 (cmt_cnt)
        cmt_cntArea1 = driver.find_elements_by_css_selector("ul.Mr508")
        cmt_cnt = len(cmt_cntArea1)
        cmt_cntList.append(cmt_cnt)

        # 글 내용 (content)
        contentArea1 = driver.find_element_by_css_selector("div.C4VMK")
        contentArea2 = contentArea1.find_element_by_tag_name("span")
        content = contentArea2.text
        contentList.append(content)

        # 해시태그 (hashtags)
        hashtags = contentArea2.find_elements_by_tag_name("a")
        tmp = []
        for i in range(len(hashtags)) :
            tmp.append(hashtags[i].text)
        hashtagsList.append(",".join(tmp))
        
    except :
        print(print((i+1), " : ", urlList[i], " fail!!!"))
        continue
    
print("Finish!!")

1  :  https://www.instagram.com/p/BN3kgnXByZ8/  start...
Finish!!


In [22]:
print(len(post_dateList))
print(len(post_tpList))
print(len(like_cntList))
print(len(view_cntList))
print(len(cmt_cntList))
print(len(contentList))
print(len(hashtagsList))

230
230
230
230
230
230
230


In [28]:
import pandas as pd

instagram = pd.DataFrame({"URL": urlList, 
                           "PostDate": post_dateList,
                           "PostTp": post_tpList,
                           "LikeCnt": like_cntList,
                           "ViewCnt": view_cntList,
                           "CmtCnt": cmt_cntList,
                           "Content": contentList,
                           "Hashtags": hashtagsList})
instagram = instagram.sort_values(by="PostDate")
instagram = instagram.reset_index()
instagram.drop(["index"], axis=1, inplace=True)

In [70]:
instagram_data = []

for i in tqdm(range(instagram.shape[0])) :
    
    instagram_data.append((TAG_NAME, 
                            instagram.loc[i, 'URL'], 
                            instagram.loc[i, 'PostDate'],
                            instagram.loc[i, 'PostTp'],
                            int(instagram.loc[i, 'LikeCnt']),
                            int(instagram.loc[i, 'ViewCnt']),
                            int(instagram.loc[i, 'CmtCnt']),
                            instagram.loc[i, 'Content'],
                            instagram.loc[i, 'Hashtags']))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


# DB 입력

In [71]:
import cx_Oracle

conn = cx_Oracle.connect("SHKIM/1234@localhost:1521/orcl")
cursor = conn.cursor()

In [72]:
query = """
SELECT *
  FROM INSTAGRAM
"""

cursor.execute(query)
print(cursor.fetchall())

[]


In [75]:
query = """
INSERT INTO INSTAGRAM
VALUES
(:1, :2, :3, :4, :5, :6, :7, :8, :9)
"""

cursor.executemany(query, instagram_data)

In [76]:
conn.commit()

In [77]:
conn.close()